In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3



In [ ]:
import gdown
url = "https://drive.google.com/file/d/18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl/view?usp=sharing"
file_id = url.split("/")[-2]
file_id

'18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl'

In [ ]:
prefix = "https://drive.google.com/uc?export=download&id="
gdown.download(prefix+file_id, "chest_data.zip")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl
From (redirected): https://drive.google.com/uc?export=download&id=18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl&confirm=t&uuid=87ae335c-081f-4aec-95a3-9d489806d76c
To: /content/chest_data.zip
100%|██████████| 124M/124M [00:02<00:00, 55.9MB/s]


'chest_data.zip'

In [ ]:
import zipfile
unzip_path = '/content'
os.makedirs(unzip_path, exist_ok=True)
with zipfile.ZipFile("chest_data.zip", 'r') as zip_ref:
  zip_ref.extractall(unzip_path)

In [ ]:
# File Directory for both the train and test
train_path = "../content/Data/train"
val_path = "../content/Data/valid"
test_path = "../content/Data/test"

In [ ]:
# Define function to count number of images per class using a dictionary
def GetDatasetSize(path):
    num_of_image = {}
    for folder in os.listdir(path):
        # count files in the folder
        num_of_image[folder] = len(os.listdir(os.path.join(path, folder)));
    return num_of_image;

# Get the number of images per class in each set (train, validation and test)
train_set = GetDatasetSize(train_path)
val_set = GetDatasetSize(val_path)
test_set = GetDatasetSize(test_path)
print(train_set,"\n\n",val_set,"\n\n",test_set)

{'normal': 148, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 115, 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 195, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 155} 

 {'normal': 13, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 21, 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 23, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 15} 

 {'normal': 54, 'squamous.cell.carcinoma': 90, 'large.cell.carcinoma': 51, 'adenocarcinoma': 120}


# DATA PREPROCESSING

In [ ]:
# Image data generator with specified augmentation configurations (mostly geometric transformations)
train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest',
                                  zoom_range=0.2,
                                  shear_range = 0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  rotation_range=0.4)

# Using data generator to create augmented data from image files in train_path directory
train_data = train_datagen.flow_from_directory(train_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')

# Dicctionary with class names to their respective indices in the generated data
train_data.class_indices

Found 613 images belonging to 4 classes.


{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1.0/255.0)
val_data = val_datagen.flow_from_directory(val_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')
val_data.class_indices

Found 72 images belonging to 4 classes.


{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.0)
test_data = test_datagen.flow_from_directory(test_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')
test_data.class_indices

Found 315 images belonging to 4 classes.


{'adenocarcinoma': 0,
 'large.cell.carcinoma': 1,
 'normal': 2,
 'squamous.cell.carcinoma': 3}

# EfficientNetB0


In [ ]:
base_model = EfficientNetB0(input_shape = (350, 350, 3),
                         include_top = False,
                         weights = 'imagenet')

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
mc = ModelCheckpoint(
    filepath="./ct_effnet_best_model.hdf5",
    monitor= 'val_accuracy',
    verbose= 1,
    save_best_only= True,
    mode = 'auto'
    );

call_back = [ mc];

tensorboard = TensorBoard(log_dir = 'logs')
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

#We also add a earlystop for prevent the computer from wasting time if it's not making progress.


early_stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

# Training the model

In [ ]:
#Customize our base model
EffNetmodel = base_model.output
EffNetmodel = tf.keras.layers.GlobalAveragePooling2D()(EffNetmodel)
# to provide overfitting problem
EffNetmodel = tf.keras.layers.Dropout(rate=0.5)(EffNetmodel)

#Finally, we add a layer with 4 'neurons' that will help us classify things into different categories.
EffNetmodel = tf.keras.layers.Dense(4,activation='softmax')(EffNetmodel)

In [ ]:
#We put together the original EfficientNetB0 and our new custom parts
EffNetmodel = tf.keras.models.Model(inputs=base_model.input, outputs = EffNetmodel)

#Compile up the rules and tools for training your neural network.
#The optimizer determines how the model should update its internal parameters, the loss function quantifies how well the model is doing, and the metrics provide additional measures to track the model's performance.
EffNetmodel.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
%time
EffNetB0 = EffNetmodel.fit(
    train_data,
    steps_per_epoch = train_data.samples//train_data.batch_size,
    epochs = 64,
    validation_data = val_data,
    validation_steps = val_data.samples//val_data.batch_size,
    callbacks = [tensorboard, mc, reduce_lr, early_stopping]
    )

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
Epoch 1/64
122/122 [==============================] - ETA: 0s - loss: 1.1243 - accuracy: 0.5082
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to ./ct_effnet_best_model.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - 72s 237ms/step - loss: 1.1243 - accuracy: 0.5082 - val_loss: 1.6983 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/64
122/122 [==============================] - ETA: 0s - loss: 0.8621 - accuracy: 0.6332
Epoch 2: val_accuracy did not improve from 0.20000


122/122 [==============================] - 26s 210ms/step - loss: 0.8621 - accuracy: 0.6332 - val_loss: 1.5721 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/64
122/122 [==============================] - ETA: 0s - loss: 0.7364 - accuracy: 0.6760
Epoch 3: val_accuracy did not improve from 0.20000

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.


122/122 [==============================] - 25s 206ms/step - loss: 0.7364 - accuracy: 0.6760 - val_loss: 2.0024 - val_accuracy: 0.0714 - lr: 0.0010
Epoch 4/64
122/122 [==============================] - ETA: 0s - loss: 0.5386 - accuracy: 0.7961
Epoch 4: val_accuracy did not improve from 0.20000


...

122/122 [==============================] - 26s 210ms/step - loss: 0.1369 - accuracy: 0.9572 - val_loss: 0.4021 - val_accuracy: 0.9000 - lr: 8.4729e-17
Epoch 61/64
122/122 [==============================] - ETA: 0s - loss: 0.1523 - accuracy: 0.9474
Epoch 61: val_accuracy did not improve from 0.92857

Epoch 61: ReduceLROnPlateau reducing learning rate to 2.541865737452411e-17.


122/122 [==============================] - 25s 207ms/step - loss: 0.1523 - accuracy: 0.9474 - val_loss: 0.4085 - val_accuracy: 0.9000 - lr: 8.4729e-17
Epoch 62/64
122/122 [==============================] - ETA: 0s - loss: 0.1470 - accuracy: 0.9507
Epoch 62: val_accuracy did not improve from 0.92857


122/122 [==============================] - 25s 206ms/step - loss: 0.1470 - accuracy: 0.9507 - val_loss: 0.3983 - val_accuracy: 0.9000 - lr: 2.5419e-17
Epoch 63/64
122/122 [==============================] - ETA: 0s - loss: 0.1389 - accuracy: 0.9556
Epoch 63: val_accuracy did not improve from 0.92857

Epoch 63: ReduceLROnPlateau reducing learning rate to 7.62559741088058e-18.


122/122 [==============================] - 25s 202ms/step - loss: 0.1389 - accuracy: 0.9556 - val_loss: 0.4096 - val_accuracy: 0.9000 - lr: 2.5419e-17
Epoch 64/64
122/122 [==============================] - ETA: 0s - loss: 0.1349 - accuracy: 0.9539
Epoch 64: val_accuracy did not improve from 0.92857


122/122 [==============================] - 25s 201ms/step - loss: 0.1349 - accuracy: 0.9539 - val_loss: 0.4085 - val_accuracy: 0.9000 - lr: 7.6256e-18


# Model Evaluation

In [ ]:
# Loading the Best Fit Model
model_eff = load_model("./ct_effnet_best_model (1).hdf5")
# Checking the Accuracy of the Model
accuracy_effnet = model_eff.evaluate_generator(generator= test_data)[1]
loss_effnet = model_eff.evaluate_generator(generator= test_data)[0]
print(f"The accuracy of the model is = {accuracy_effnet*100} %")
print(f"The loss of the model is = {loss_effnet} %")

<ipython-input-18-2f35d78c66ee>:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  accuracy_effnet = model_eff.evaluate_generator(generator= test_data)[1]
<ipython-input-18-2f35d78c66ee>:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_effnet = model_eff.evaluate_generator(generator= test_data)[0]


The accuracy of the model is = 94.60317492485046 %
The loss of the model is = 0.2430509328842163 %
